# Classification with neural networks / ニューラルネットワークによる分類

The picture bellow shows a biological neuron (top) and an artifical neuron (bottom)

下の写真は、生物学的のニューロン(神経細胞；上）と人工ニューロン（下）を示しています。

<br>
<img src="./img/neurons.jpg" width="400">
<br>

- The "inputs" of the artificial neuron act like the dentrites of the biological neurons.
- The "output" of the artificial neuron act like the axon of the biological neurons.
- The artificial neuron computes a ponderated sum of the inputs and outputs a non linear function of that sum.
<br><br>

- 人工ニューロンの「入力」は、生物学的ニューロンの樹状突起のように作用します。
- 人工ニューロンの「出力」は、生物学的ニューロンの神経繊維（軸索）のように作用します。
- 人工ニューロンは入力の加重和を計算し、その和の非線形関数を出力します。


The operation of the artificial neuron can be written with the following equation:

人工ニューロンの動作は次の式で書くことができます。

$y = f(\mathbf{W} \mathbf{X} + b)$

The ponderation of the inputs ($\mathbf{W}$) are referred to as the __weights__ and the non linear function $f$ the __activation__.

入力の加重$\mathbf{W}$は__重み__と呼ばれ、関数$f$は__活性化関数__と呼ばれます。$f$は主に非線形関数です。

Like biological neurons, the "power" of artificial neuron is the result of combining many neurons in a network. The resulting structure is called an __artificial neural network__ (ANN) or often simply a __neural network__ (NN).

生物学的ニューロンと同様に、人工ニューロンの「強み」は、多数のニューロンを組み合わせてネットワークにすることから生じる。得られた構造は__人工ニューラルネットワーク__と呼ばれます。省略して__ニューラルネットワーク__と呼ばれることがよくあります。

<img src="./img/network.jpg" width="400">


In this session, we will see how to build artificial neural networks and use them for classifying the digit of the MNIST dataset and the flowers of the iris dataset.

このセッションでは、人工ニューラルネットワークを構築し、MNISTデータセットやアヤメのデータセットを分類することを使用する方法について説明します。

### Preparing the tools / ツールの準備

First, let us import the usual packages.

まず、必要なパッケージをインポートしましょう。

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

Let us import packages for using neural networks. `tensorflow` is a DNN library that also makes possible to perform computations on the GPU to accelerate the computing.

ニューラルネットワークを使用するためのパッケージをインポートしましょう。 `tensorflow`は計算を加速するためにGPU上で計算を実行することも可能にするDNNライブラリです。

In [ ]:
import tensorflow as tf

Here we will not `tensorflow` directly. Instead we will use a frontend library called `keras`, which allows easy building of neural networks.
<br>
The lines below import all the building blocks for the networks that we will create in this session.

ここでは`tensorflow`は直接使いません。代わりに`keras`と呼ばれるフロントエンドライブラリを使用します。`keras`を使えばニューラルネットワークを簡単に構築することができます。
<br>
以下の行は、このセッションで作成するネットワークのすべての構成要素をインポートしています。

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical

### Load MNIST dataset / MNISTデータセットを読み込む

This time, we load the full MNIST dataset (70000 samples).

先週のセッションと違って、今回はMNISTデータセットの全体（70000サンプル）をロードします。

In [ ]:
from mnist_loader import MNISTVectorLoader
# 43 is used to initialize the random generator in the object
mnist_vector_loader = MNISTVectorLoader(43)
X, y = mnist_vector_loader.samples(70000)

Split the dataset between training and testing sets.

データセットをトレーニング用のセットとテスト用のセットに分割します。

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.5)

print("Training set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])

Apply standardization: 

標準化を適用します

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Create a fully connected network / 全結合ネットワークを作成する

First, let us define a placeholder for the input of the network using the object `Input` from `keras`.
<br>
We get the `shape` parameter value from one sample of the training set.

まず、`keras`の`Input`オブジェクトを使って、ネットワークの入力用のプレースホルダーを定義しましょう。<br>
`shape`パラメータ値はトレーニング（訓練）セットの1つのサンプルから取得します。

In [ ]:
input_shape = X_train[0].shape
print("Input shape:", input_shape)
vector_input = Input(shape = input_shape, name='input')

The input placeholder defines an "input line" for each of the `784` values of a training sample.
<br>
Let us now create the first layer of the network.It is a layer of neurons that are fully connected (fc) to the "input lines".
<br>
Namely, each of the neurons of the layer `fc1` are connected to all of the input lines provided by the input placeholder.

入力プレースホルダは、トレーニングセットの各784サンプルの「入力線」を定義する。
<br>
次は、ネットワークの最初の層を作成しましょう。全ての「入力線」に完全に接続されているニューロンの層、いわゆる__全結合層__です。
<br>
すなわち、層`fc1`の各ニューロンは、入力プレースホルダのすべての入力線に接続される。

In [ ]:
fc1 = Dense(128, activation='relu', name='fc1')(vector_input)

The layer `fc1` is of type `Dense`. In `keras`, a fully connected layer is called a `Dense` layer. It has 128 neurons.
<br>
The activation (nonlinear function) is of type `relu`. This is a simple and most commonly used activation function called _Rectified Linear Unit (ReLU)_. The output of ReLU is the same as the input if the input is positive, and 0 if it is negative.

Note that the layer `fc1` is a function of the layer `vector_input`. The input placeholder `vector_input` provides the inputs of the layer `fc1`. This is the way the `keras` functionnal API connects the layers to build a network.
<br>
The layer `fc1` has a weight for all the connections between the 784 input lines and the 128 neurons. Thus, there are `784x128 = 100480` weights.

`fc1`層のタイプは` Dense`です。`keras`では、全結合層は` Dense`層と呼ばれます。128ニューロンを持っています。
<br>
使っている非線形関数（_活性化関数_）は` relu`型です。これは、_正規化線形関数（Rectified Linear Unit：ReLU）_と呼ばれるシンプルで最も一般的に使用される活性化関数です。ReLUの出力は、入力が正の場合は入力と同じであり、負の場合は0です。

`fc1`層は` vector_input`の関数です。すなわち、入力プレースホルダ `vector_input`は` fc1`層の入力を提供します。このようにして、 `keras`APIでは層を接続してネットワークを構築します。
<br>
層`fc1`には、784の入力線と128のニューロンがあって、その間のすべての接続に重みをあります。したがって、重みが全部で`784x128 = 100480`あります。

Let us add another fully connected layer with 128 neurons:

128個のニューロンを持つ別の全結合層を追加しましょう。

In [ ]:
fc2 = Dense(128, activation='relu', name='fc2')(fc1)

This new layer `fc2` also of type `Dense` takes the previous layer `fc1` as input.

この`fc2`層も`Dense`型です。前の`fc1`を入力として使っています。

At this stage, the neural network is composed of two layers of `128` neurons.
<br>
When an input vector of size `784` is presented to the network by the input placeholder:
- `fc1` computes `128` weighted sums of these `784` values and apply the `relu` function to each of these sums.
- `fc2` computes `128` weighted sums of the `128` outputs of `fc1` and apply the `relu` function to each of these sums.

この段階で、ニューラルネットワークは2層があって、それぞれの層に128のニューロンがあります。
<br>
サイズ784の入力ベクトルが入力プレースホルダ経由でネットワークに入れると、
 -  `fc1`はこれらの784値から128の加重和を計算し、各和に`relu`関数を適用します。
 -  `fc2`は`fc1`の128の出力から128の加重和を計算し、それぞれの和に`relu`関数を適用します。

Let us add another fully connected layer for the output: 

ネットワーク出力になるもう1つの全結合層を追加しましょう。

In [ ]:
output = Dense(10, activation='softmax', name='output')(fc2)

This new layer `output` is also a fully connected layer. However it has only `10` neurons and the activation is no longer of type `relu`.
<br>
The activation function `softmax` is a function with multiple inputs and multiple outputs, in this case, `10` of each.
<br>
It is defined by the equation below.

この`output`層も全結合層ですが、10個のニューロンしか持っていません。
<br>
また、活性化関数は`relu`型ではなく、ソフトマックス関数（`softmax`）です。
<br>
以下は`softmax`関数の定義です。

$\text{softmax}\left(i_0, i_1, \cdots, i_{n-1}\right) = \left(\frac{\exp{i_0}}{\sum_k \exp{i_k}}, \frac{\exp{i_1}}{\sum_k \exp{i_k}}, \cdots, \frac{\exp{i_{n-1}}}{\sum_k \exp{i_k}}\right) $
 
The outputs of the softmax function take values between 0 and 1, and their sum is 1.

`softmax`の出力は０から１の間の値を取り、それらの合計は１です。

The number of outputs of the `output` layer corresponds to the number of digits to classifiy in the MNIST dataset. For a given input, one of these outputs will have the highest value. The digit to which this output corresponds is considerd to be the result of the classification. (E.g. if the outputs from `output` are [0, 0, 0.7, 0, 0, 0, 0.2, 0, 0.1, 0], the third one is the largest, which corresponds to the digit `2`).
<br>
Using this way to determine the resulting class, the network is able to perform the digit classification. 
<br>
The network still needs to "learn" . The goal of this learning is to find adequate values for the weights of `fc1`, `fc2` and `output` such that when presented with a picture of a given digit, the network predicts the correct value of the digit by activating the corresponding output of `output`.

`output`層の出力の数は、MNISTデータセットで分類する桁数に対応します。ネットワークにある入力を与えられたら、`output`層のどちらか1つの出力が最も高い値を持ちます。その出力に対応する数字は分類の結果であると見なされます。（例えば、`output`層からの出力が[0, 0, 0.7, 0, 0, 0, 0.2, 0, 0.1, 0]だとすると、その3番目の値は最大であり、それが数字「2」に対応する）
<br>
今のやり方で結果クラスを決定して、ネットワークが手書き数字の分類を実行できます。
<br>
ネットワークはまだ「学習」する必要があります。この学習によって、`fc1`、`fc2`と`output`層の重みの適切な値を見つかります。すると、手書き数字の図が与えられたら、その数字に対応する`output`の出力をアクティブになるようになります。


Now, we should tell `keras` that we are done with creating our network.

これでネットワーク構造の定義が終わったため、ネットワークを作成しましょう。

In [ ]:
network = Model(vector_input, output, name='classification')

The `Model` function creates the network (or model) by indicating the input and the output, respectively `vector_input` and `output`.

Model関数はネットワーク（モデル）を作成します。入力と出力がそれぞれ`vector_input`と`output`と指示します。

The `keras` models have a convenient function to display their parameters:

`keras`のモデルのパラメータを表示するには以下の関数を使えます。

In [ ]:
network.summary()

The `summary` function displays all the information about the network layers.
<br>
The value `None` in the shapes indicates that any size is possible. For example the input can be a single sample of size `(1, 784)` or our whole training set of size `(35000, 784)`.
<br>
The summary indicates the number of parameters that are trainable. This particular model has `118,282` that are all trainable.
<br>
As expected, the layer `fc1` that provides `128` weighted sums of the `784` inputs has the most parameters (`784x128=100480`).

`summary`関数は、ネットワークの層に関する情報を表示します。
<br>
形状の値が`None`の場合、任意のサイズが使えることを示します。 たとえば、入力にはサイズ（1、784）の単一のサンプルも使えて、サイズ（35000、784）の全トレーニングセットも使えます。
トレーニング可能なパラメーター（重み）の数も示しています。 このモデルはパラメータを118,282を持って、すべて訓練可能なパラメータです。
その中、784の入力の128の加重和を提供するレイヤー`fc1`が最も多くのパラメーターを持っています（784 x 128 = 100480）。

---
Note: The network we created has only 3 layers. Nowadays neural networks with a much larger number of layers (hundreds or even thousands) are often used. These networks are usually called _Deep Neural Networks (DNN)_. The classification using DNNs is often reffered to as _Deep Learning_.

注：作成したネットワークは3つの層しかありません。今日では、はるかに多数の層（数百、さらには数千）を持つニューラルネットワークがよく使用されます。これらのネットワークは通常_ディープニューラルネットワーク（DNN）_または_深層ネットワーク_と呼ばれます。 DNNを使用した分類は、多くの場合_ディープラーニング_または_深層学習_と呼ばれています。

---

## Try  it yourself ! / 自分で試そう！

[Click here](session2-playground1.ipynb) to open a sample notebook where you can create your own model.

[ここをクリックして](session2-playground1.ipynb)練習ノートブックを開いて、自分でニューラルネットワークを構築してみてください。

### Training the network / ネットワークの訓練

The MNIST labels are digits:

MNISTのラベルは数字です。

In [ ]:
print(y_train[:10])

Before using our dataset, we have to transform the label so that `keras` can compare them to the output of `output`.
The cost function has to compare the label to the outputs of the softmax function in `output`, which is a vector of size `10`.
<br>
Consequently we need to perform a __"one-hot" encoding__ of the labels in order to transform a label in a vector of size `10`.

The one-hot encoding is a vector having a length equal to the number of classes; here `10`.
<br>
The elements of the vector are set to `0` except for the position corresponding to the encoded class that is set to `1`.
<br>
For example for the digit is `5` the one-hot encoding is `[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]`

During training, the one-hot encoding of the digit is the compared to the output of the network.


データセットを使う前に、 `keras`がそれらを` output`の出力と比較できるようにラベルを変換する必要があります。
コスト関数はサイズが10のベクトルである`output`の中のsoftmax関数の出力とラベルを比較しなければなりません。
<br>
そのため、ラベルを10の大きさのベクトルに変換する必要があります。ラベルをベクトルに変更するには__one-hot(ワンホット)エンコーディング__を使用します。

ワンホットエンコーディングは、クラス数に等しい長さを持つベクトルです。
<br>
ベクトルの要素は、対象クラスに対応する要素のみが`1`に設定され、残りは`0`に設定される。
<br>
例えば、数字が`5`の場合、ワンホットエンコーディングは`[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]`です。

トレーニング中、数字のワンホットエンコーディングはネットワークの出力と比較されます。

The function `to_categorical` from `keras.utils` does this encoding:

ワンホットエンコーディングを行うためには`keras.utils`の`to_categorical`関数が使えます。

In [ ]:
y_train_one_hot = to_categorical(y_train)

Let us look at a few labels:

得られたラベルをいくつか見てみましょう。

In [ ]:
for i in range(10):
    print("Original digit label:", y_train[i], "Created one-hot vector label:", y_train_one_hot[i, :])

Now that the labels are in the right format, let us prepare the training.

ラベルが正しい形式になったので、トレーニングの準備に移ります。

#### Defining the algorithm for training / トレーニング用アルゴリズムの定義

The `keras` library contains many algorithms for training neural networks.
<br>
These algorithms are __optimization algorithms__ that _adjust the trainable parameters of a model in order to minimize a loss function_.

In the case of classification, while training the network with known examples from the training set, the __cost function__ measures how "close" is the predicted output of the network to the true value.
<br>
Then, the optimizer updates the trainable weights in order to make the predicted output a "bit closer" to the true value.

`keras`ライブラリはニューラルネットワークを訓練するための多くのアルゴリズムを持っています。
<br>
これらのアルゴリズムは __最適化アルゴリズム__ です。モデルの _トレーニング可能なパラメータを調整し損失関数を最小化_ します。

分類の場合、トレーニングセットのサンプルを用いてネットワークを訓練している間、__コスト関数__ はネットワークの出力が真の値にどれだけ「近い」かを測定する。
<br>
最適化アルゴリズムは予測出力を真の値に「少し近づける」ようにネットワークの重みを更新します。

In `keras`, to use an optimizer, it is necessary to:
- define the loss function, also called `loss`
- choose the optimizer
- select performance metrics

This is done by calling the `compile` method of the `Model` object:

`keras`では、オプティマイザを使うためには、次のことが必要です。
 - 損失関数（`loss`）を定義する
 - オプティマイザを選択
 - パフォーマンス指標を選択
 
これは`Model`オブジェクトの` compile`メソッドを呼び出すことによって行われます。

In [ ]:
network.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['acc'])

The parameters we used here are very typical. In most cases this will work well for classification using typical neural networks.

- The __categorical crossentropy__ loss function (`categorical_crossentropy`) is the basic choice for classification with multiple classes.
- The optimizer is of type `Adam`, and the parameter `lr` controls how "fast" the weights are modified. (`keras` provides several different optimizers but we will not explain them here in detail. `Adam` usually performs well.)
- The perfromance petrics that we are trying to optimize is accuracy (`acc`).

ここに使用したパラメータはかなり典型的なものです。ほとんどの場合、一般的なニューラルネットワークを使用した分類にこのパラメータが適しています。

- 複数のクラスでの分類には一般 __多クラス交差エントロピー__ `categorical_crossentropy`損失関数を使います。
- オプティマイザは`Adam`型で、パラメータ`lr`は重みがどのくらい速く変更されるかを制御します。（`keras`は複数のオプティマイザを提供します。ここではオプティマイザについて詳細に説明しません。`Adam`は通常、うまく機能します。）
- 最適化しようとしているパフォーマンス指標は精度（`acc`）です。

After compiling, the model is ready for training.

これでトレーニングの準備ができています。

#### The network training / ネットワークのトレーニング

The `Model` object provides the `fit` method to handle the training.
<br>
A basic use of `fit` takes the following parameters:
- a set of features: `X_train`
- the corresponding labels: `y_train_one_hot` 
- a batch size: `batch_size`
- a number of epochs: `epochs`
- a validation set: `validation_data` 

`Model`オブジェクトはトレーニングを行う` fit`メソッドを提供します。
<br>
`fit`の基本的な使い方には以下のパラメータが必要です：
 - 入力データ（特徴）： `X_train`
 - 対応するラベル： `y_train_one_hot`
 - バッチの大きさ： `batch_size`
 - エポックの数： `epochs`
 - 検証（テスト）データセット： `validation_data`

`batch_size` and `epochs` control how the optimizer handles the samples:
- The batch size defines the number of consecutive samples that are used to estimate the cost function and update the weights.
- The optimizer has performed one epoch when it has processed all the samples (it went hrough the dataset one time).

For example, if the training set has 35000 samples, the batch size is 100 and `epochs` is set to 20, then the optimizer goes throught the whole `35000` samples `20` times and each time updates the weights `350` times using loss values computed from batches of `100` samples.

`batch_size`と` epochs`はオプティマイザがサンプルをどのように扱うかを制御します。
 - バッチサイズは、コスト関数の推定と重みの更新に使用される連続サンプル数を定義します。
 - １エポックというのは、オプティマイザがすべてのサンプルの処理を1回実行しました（データセットを1回通過しました）。

例えば、トレーニングセットが35000サンプルを有する場合、バッチサイズは100であり、`epochs`は20に設定されたにしましょう。オプティマイザは35000サンプル全体を20回通過します。それぞれのエポックに350回重みを更新します。それぞれの更新には100サンプルのバッチから計算された損失値を使用します。

The optional parameter `validation_data` is a dataset composed of features and associated labels.
<br>
At the end of each epoch the loss function and the accuracy is computed for the validation set.
<br>
Namely, if we provide the testing set as `validation_data`, the performance on the testing set will be estimated after each epoch.
<br>
To do that, we need to encode the test set labels as one-hot vectors:

オプションのパラメータ `validation_data`は入力データ（特徴）とそれに関連するラベルからなるデータセットです。
<br>
各エポックの終わりに、検証セット上の損失関数と精度が計算されます。テストセットを`validation_data`として提供すれば、テストセットのパフォーマンスは各エポックの後に推定されます。そのためには、テストセットのラベルに予めワンホットエンコードを実行する必要があります。

In [ ]:
y_test_one_hot = to_categorical(y_test)

Now, let us do the actual training of the network. We run the `fit` method to train `network`. The training takes some time so be patient. 

それでは、実際にトレーニングを行いましょう。`fit`メソッドを実行して`network`を訓練します。
訓練にはしばらく時間がかかりますので辛抱してください。

In [ ]:
H = network.fit(X_train, y_train_one_hot, batch_size=100, epochs=20, validation_data=(X_test, y_test_one_hot))

The `fit` method reports for each epoch:
- the total computation time of the epoch (rounded)
- the computation time per samples
- the value of the loss function (on the training set)
- the accuracy of the network (on the training set)
- the value of the loss function (on the validation set)
- the accuracy of the network (on the validation set)

各エポックについて`fit`メソッドが以下を報告します：
 - エポックの合計計算時間（四捨五入）
 - サンプルあたりの計算時間
 - （トレーニングセット上の）損失関数の値
 - （トレーニングセット上の）精度
 - （テストセット上の）損失関数の値
 - （テストセット上の）精度

The value of the loss function is the important parameter here.
<br>
If the training is going well, the loss function should decrease.

ここで一番重要なパラメータは損失関数の値です。
<br>
訓練がうまく行けば、損失関数は減少するはずです。

The output of the `fit` method (which we assign to the object `H`) contains a detailed report of the training process.
<br>
In particular `H.history` is a python dictionary containing the evolution of the loss function and accuracy. 
<br>
It is possible to plot the evolution of the loss function and the accuracy.

`fit`メソッドの出力はトレーニング過程の詳細なレポートを含みます。（上のコーデセルではこれを`H`に入れています）
<br>
`H.history`には損失関数と正確さの進化が入っていますので、損失関数や精度の進化をプロットすることができます。

Let us first look at the loss function:

まず損失関数を見てみましょう。

In [ ]:
plt.plot(H.history['loss'], 'o-',label="loss")
plt.plot(H.history['val_loss'], 'o-', label="val_loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("loss vs epochs")
plt.legend();

#### Overfitting

We can see that the loss function for the training set (blue line) is steadily decreasing whereas the loss function for the testing set (orange line) reaches a minimum and then stays the same or increases a bit.
This is a sign of __overfitting__.
<br>
Overfitting occurs when the classifier starts to learn the specific peculiarities of the samples in the training set. However, by doing that it does not fit well to other data anymore. 
<br>
After the tenth epoch, the network is still learning to better fit the training set. But the featues that are being learned are specific only to the training set and are thus not useful for classifying the testing set.

トレーニングセットの損失関数（青い線）は着実に減少していますが、テストセットの損失関数（オレンジ）は最小値に達してから、同じままか少し増加してことがわかります。これは__過学習__の証拠です。
<br>
過学習は、分類器がトレーニングセット内のサンプルの特定の特性をさらに学習するが、他のデータには適合しなくなる現象です。
<br>
10回目のエポックの後、ネットワークはまだトレーニングセットによりよく適合するように学習しています。 ただし、学習される機能はトレーニングセットにのみ固有であるため、テストセットの分類には役立ちません。

Let us plot the accuracy:

精度もプロットしてみましょう。

In [ ]:
plt.plot(H.history['acc'], 'o-', label="acc")
plt.plot(H.history['val_acc'], 'o-', label="val_acc")
plt.xlabel("epochs")
plt.ylabel("acc")
plt.title("Accuracy vs epochs")
plt.legend();

We can also see that the accuracy is no longer improving after the tenth epoch.

10回目のエポック以降、精度もほとんど向上しなくなったことがわかります。

#### Preventing overfitting / 過学習の防止

One simple method to prevent overfitting is to _stop the training early_. From the curves above, it seems it would be better to stop the training after 10 epochs. 
<br>
The `fit` method provide a convenient way to do that; it accepts `callbacks`.
`callbacks` are functions that are called at specific timings during the training.
<br>
`keras` defines a callback called `EarlyStopping` that does just what we want - it can stop the training.

過学習を防ぐため、1つの簡単な方法は _訓練の中断_ です。例えば、上の曲線を見れば、10エポックの後にトレーニングを中止したら良かったと思われます。
<br>
`fit`メソッドには`コールバック`関数が使えます。（「コールバック」というのは実行中の特定のタイミングで呼び出される関数です。）
<br>
訓練を止めるには`EarlyStopping`というコールバックが使えます。

Here we define the callback for stopping early.

トレーニングを早めに中断するためのコールバックを定義します。

In [ ]:
early_stopping_cb = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2, verbose=1, mode='auto')

At the end of each epoch, this callback checks if the loss function for the validation set has decreased of at least `min_delta`.
<br>
If for `patience` times it has not decreased, the training is interrupted.

各エポックの終わりに、このコールバックはテストセット上の損失関数が`min_delta`以上減少したかどうかチェックします。
<br>
連続的に減少していない回数が`patience`以上になりますと、訓練は中断されます。

#### Saving best models / 最高のモデルの保存

Another useful callback is `ModelCheckpoint` that saves the model with the best performance (on the validation data) to a file.

もう1つの便利なコールバックは `ModelCheckpoint`です。これは（テストデータに対して）最高のパフォーマンスのモデルをファイルに保存します。

In [ ]:
model_checkpoint_cb = ModelCheckpoint("best_network.hdf5", monitor='val_loss', verbose=1, 
                                      save_best_only=True, save_weights_only=False, mode='auto', period=1)

This callback monitors `val_loss` and saves only the best model (model structure and trained weights) in the file `best_network.hdf5`.

このコールバックは`val_loss`を監視し、その値が最低になったモデル（ネットワークの構造と重み）をファイル` best_network.hdf5`に保存します。

Then, we just have to provide the callback objects to the `fit` method.
<br>
The function `fit` expects a list of callbacks. 
Here we provide a list of two elements.

それから、コールバックオブジェクトを `fit`メソッドに渡すだけです。
<br>
`fit`はコールバックのリストを期待します。ここでは2つの要素のリストを提供します。

In [ ]:
H = network.fit(X_train, y_train_one_hot, batch_size=100, epochs=20, validation_data=(X_test, y_test_one_hot),
                callbacks=[early_stopping_cb, model_checkpoint_cb], verbose=1)

You can see that `fit` reports saving the model and stops before completing all the epochs.

`fit`はモデルの保存を報告し、すべてのエポックを完了する前に停止することがわかります。

####  Reloading the model / 保存したモデルの再読み込み

The `ModelCheckpoint` callback saved the network with the lowest `val_loss` in the file `best_network.hdf5`.
<br>
It is possible to load the weights of that `model` with the following command.

`ModelCheckpoint`コールバックはファイル` best_network.hdf5`の最も低い `val_loss`でネットワークを保存しました。
<br>
次のコマンドでその`model`の重みをロードすることができます。

In [ ]:
network.load_weights('best_network.hdf5')

Now the weights in `network` are set to the weights that were saved in `best_network.hdf5`.

これで`network`の重みは`best_network.hdf5`に保存された重みに設定されました。

It is also possible to create a new `Model` using the function `load_model` from the package `tensorflow.keras.models`.

パッケージ`tensorflow.keras.models`にある`load_model`関数を使えば新しい`Model`を作成することもできます。

In [ ]:
from tensorflow.keras.models import load_model
best_network = load_model('best_network.hdf5')

After this command `best_network` is a newly created `Model` identical to the best `Model` that was created during training.

このコマンドの後、`best_network`はトレーニング中に作成された最高の`Model`と同一の`Model`です（構造も重みも同じです）。

During the training, the `ModelCheckpoint` callback saved the network.
We can also save a `Model` directly by using the `save` method.

トレーニング中、`ModelCheckpoint`コールバックはネットワークを保存しました。
`save`メソッドを使えば`Model`を直接保存することも可能です。

In [ ]:
network.save('my_network.hdf5')

####  Performance of the network / ネットワークの性能

The `predict` method of a `Model` gives the predicted output for the given input.

`Model`の` predict`メソッドは与えられた入力に対する出力の予測を返します。

In [ ]:
y_pred_one_hot = best_network.predict(X_test)

The network prediction is the values of the output.
<br>
Here, it is a vector of length dimension `10`.

ネットワーク予測は、出力の値です。
<br>
ここでは、長さ次元`10`のベクトルです。

In [ ]:
y_pred_one_hot[0,:].shape

We used the `softmax` activation function in the output layer of the network. The result shows it is dominated by a single value.

ネットワークの出力層で`softmax`活性化関数を使いました。予測結果を見ると、多くの場合一つの値だけが大きくなっています。

In [ ]:
plt.bar(np.arange(10), y_pred_one_hot[0,:])
plt.xlabel("index")
plt.ylabel("value")
plt.title("Network Output");

If we display a few samples as an image, we can see that most of the rows are dominated by a single value.

いくつかのサンプルの予測結果を画像として表示すると、ほとんどの行では一つの値だけが大きいことがわかります。

In [ ]:
plt.imshow(y_pred_one_hot[:20,:])
plt.xlabel("output vector")
plt.xticks(np.arange(10))
plt.ylabel("samples")
plt.title("predictions")
cbar = plt.colorbar()

The function `argmax` from `numpy` gives the index corresponding to the maximum of a numpy array.

`numpy`の`argmax`関数は、numpy配列の最大値に対応するインデックスを返します。

In [ ]:
print("index of the max:", np.argmax(y_pred_one_hot[0,:]))

By specifying the axis along which to find the maxima (here `axis=1`), it can be applied to the complete prediction block.

検索軸を指定すれば（ここでは `axis = 1`）、複数の予測結果の最大値を計算することができます。

In [ ]:
y_pred = np.argmax(y_pred_one_hot, axis=1)

Now, `y_pred` contains values in `[0,9]`. These are the predicted values for the digits in the input image.

`y_pred`には0から9の値が入っています。入力画像の数字の予測です。

In [ ]:
plt.plot(y_pred[:20], 's')
plt.xlabel("samples")
plt.ylabel("predicitions");

It is possible to use the `sklearn` functions to check the performance, like we did last week.

先週やっていたように、`sklearn`の数を使ってパフォーマンスを確認できます。

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
CM = confusion_matrix(y_test, y_pred)
print(CM)
acc = accuracy_score(y_test, y_pred)
print(acc)

In [ ]:
plt.imshow(CM)
plt.xlabel("Predicted digit")
plt.xticks(np.arange(10))
plt.yticks(np.arange(10))
plt.ylabel("True digit")
plt.title("True vs predicted digits")
cbar = plt.colorbar()
cbar.ax.set_title("#samples", rotation=0);

## Try  it yourself ! / 自分で試そう！

Since the network created here is taking GPU resources, you should first free them to create networks in other Jupyter notebooks.
<br>
One easy way to do this is by shuting down or restaring the kernel in this notebook.
<br>
Run now the menu command `Kernel`->`Restart`.

ここで作成されたニューラルネットワークはGPUリソースを使用しています。他のJupyterノートブックでネットワークを作成するためには、まずGPUリソースを解放する必要があります。
<br>
これを行う簡単な方法の1つは、このノートブックでカーネルを停止または再起動することです。
<br>
今すぐメニューコマンド`Kernel`->`Restart`を実行してください。

Then, [click here](session2-playground2.ipynb) to open a sample notebook.

次に、[ここをクリックして](session2-playground2.ipynb)、サンプルのノートブックを開きます。